# **Adding New Data with slides:**

## Process

1. **blnk_update_dt_row_dict** is used to collect data0
2. **Select "dtv"s for adding data** *done in main_workbook*
3. **load_lut_config(csv_path) to set defaults etc** collects for csv.file.
4. *Collects data on specified* **dtv**s
5. *Shows sliders of the default dosage:
6. Then operatoradjust them as to reflect changes from normal;
7. Then put those values into **blnk_update_dt_row_dict**
8. When **"On submit"** **"update_dt_row_dict** is created
9. Then loaded into* **update_dt_row_dict.pkl**
10. Then verify the changes
11. Adjust **dt_row_dict** with the data from **update_dt_row_dict**

## **Update**

### **Imports**

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pickle

### **Loaded DTV range**

In [2]:
##### import "dtv" from pickle
with open("dtv_input.pkl", "rb") as f:
    dtv = pickle.load(f)
print("Loaded DTV range:", dtv)


Loaded DTV range: [45861, 45861]


In [3]:
dtv

[45861, 45861]

### **blnk_update_dt_row_dict**

In [4]:
# Load blank  master dt_row_dict template all ""
with open("blnk_update_dt_row_dict.pkl", "rb") as f:
    blnk_update_dt_row_dict = pickle.load(f)
blnk_dt_row_dict = blnk_update_dt_row_dict

In [5]:
blnk_dt_row_dict           #  master dt_row_dict template all ""

{'45861': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45862': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45863': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45864': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45865': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

### **blnk_update_dt_row_dict** *select only specified* **"dtv"s** *for update*

In [6]:
# def select_dtv_rows(dtv_range, blnk_dt_row_dict):
# this the part of main blank dict we want to update.
def select_dtv_rows(dtv_range, source_dict):
    print(dtv_range)
    print(blnk_dt_row_dict)
    # Ensure keys are sorted numerically
    sorted_keys = sorted(source_dict.keys(), key=int)

    # Unpack and normalize range
    start, end = sorted(dtv_range)
    
    # Filter keys within range
    selected = {
        k: source_dict[k]
        for k in sorted_keys
        if start <= int(k) <= end
    }
    return selected

# Example usage
# dtv = [45962, 45864]  Set ealier
blnk_update_dt_row_dict = select_dtv_rows(dtv, blnk_dt_row_dict)

#selected_rows = select_dtv_rows(dtv, blnk_my_dt_row_dict)
#print(selected_rows)
#

[45861, 45861]
{'45861': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}, '45862': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}, '45863': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}, '45864': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}, '45865': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}


In [7]:
blnk_update_dt_row_dict 

{'45861': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

### **Load attributes from luts** *and preset the* **"Slider"s** 

In [8]:
# 🔹 Step 1: Load Inputs
import pickle
import csv

# Load blank update dictionary
#with open("blnk_update_dt_row_dict.pkl", "rb") as f:
#    blnk_update_dt_row_dict = pickle.load(f)

blnk_update_dt_row_dict 
# Load and parse lut_serial_config.csv
def load_lut_config(csv_path):
    with open(csv_path, newline='') as f:
        reader = list(csv.reader(f))
    headers = reader[0][1:]  # ['s000', 's001', ...]
    lut = {}
    
    for row in reader[1:]:
        key = row[0].strip('"')
        for i, val in enumerate(row[1:]):
            serial = headers[i]
            lut.setdefault(serial, {})[key] = val.strip('"')
    # Convert types
    for serial, config in lut.items():
        config['value'] = int(config.get('value', 1))
        config['min'] = int(config.get('min', 0))
        config['max'] = int(config.get('max', 10))
        config['step'] = int(config.get('step', 1)) if config.get('step') else 1
        config['active'] = config.get('active', 'true').lower() in ['true', 'yes']
    return lut

lut_serial_config = load_lut_config("lut_serial_config.csv")


In [9]:
blnk_update_dt_row_dict

{'45861': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

### *Pre=adjusted* **Sliders Presented** *make changes if the slider values are wrong, then* **"Submit"** 

In [10]:
# Step 2: Build Slider Form
import ipywidgets as widgets
from IPython.display import display, clear_output

update_dt_row_dict = {day: dict(serials) for day, serials in blnk_update_dt_row_dict.items()}
output = widgets.Output()
display(output)

form_items = []
slider_widgets = {}

for day, serials in update_dt_row_dict.items():
    form_items.append(widgets.HTML(value=f"<b>Day: {day}</b>"))
    for serial in serials:
        config = lut_serial_config.get(serial, {})
        if not config.get('active', True):
            continue
        slider = widgets.FloatSlider(
            value=config['value'],
            min=config['min'],
            max=config['max'],
            step=config['step'],
            description=f"{config['name']}:",
            layout=widgets.Layout(width='400px')
        )
        slider_widgets[(day, serial)] = slider
        form_items.append(slider)

submit_button = widgets.Button(description="Submit")

def on_submit(b):
    for (day, serial), slider in slider_widgets.items():
        update_dt_row_dict[day][serial] = slider.value
    with open("update_dt_row_dict.pkl", "wb") as f:    
        pickle.dump(update_dt_row_dict, f)
    with output:
        clear_output()
        print("✅ Update complete. Result saved to [[update_dt_row_dict.pkl]]")
        for day, serials in update_dt_row_dict.items():
            print(f"{day}: {serials}")

submit_button.on_click(on_submit)
form_items.append(submit_button)
form = widgets.VBox(form_items)

with output:
    clear_output()
    print("Adjust sliders for each active supplement. Suspended ones are skipped.\n")
    print("update_dt_row_dict is loaded'; go to\n ### *update* *the* **dt_row_dict**  *with the values from* **update_dt_row_dict** ")

    display(form)


Output()

### *Verify changes and update* *the* **dt_row_dict**  *with the values from* **update_dt_row_dict**

In [11]:
with open("update_dt_row_dict.pkl", "rb") as f:
    update_dt_row_dict = pickle.load(f)
    print(update_dt_row_dict)

{'45861': {'s000': 1.0, 's001': 1.0, 's002': 2.0, 's003': 1.0, 's004': 1.0}, '45862': {'s000': 1.0, 's001': 1.0, 's002': 2.0, 's003': 1.0, 's004': 0.0}}
